In [0]:
from discoverx import DX
dx = DX()

In [0]:
from_tables = "*.*.*"

In [0]:
table_counts=dx.from_tables(from_tables).with_sql("""SELECT COUNT(*) as count, current_date() as event_date FROM {full_table_name}""").apply()
table_counts= table_counts.filter("table_catalog not in ('samples')")
table_counts.display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS quality.default.table_counts
(
    count LONG,
    event_date DATE,
    table_catalog STRING,
    table_schema STRING,
    table_name STRING
)

In [0]:
spark.sql("""
MERGE INTO quality.default.table_counts AS target
USING {table_counts} AS source
ON target.table_catalog = source.table_catalog
   AND target.table_schema = source.table_schema
   AND target.table_name = source.table_name
   AND target.event_date = source.event_date
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *
""",table_counts=table_counts)

In [0]:
%sql
CREATE OR REPLACE TABLE quality.default.table_counts_current AS
with row_num as (
SELECT *, row_number() OVER(PARTITION BY concat(table_catalog,'.',table_schema,'.',table_name) ORDER BY event_date DESC) as rn FROM quality.default.table_counts QUALIFY rn = 1)
select * EXCEPT(rn) from row_num